In [ ]:
# Activar Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Calcula el pago de luz para tarifas residenciales.

import pandas as pd
import numpy as np

In [ ]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/"

# Escenarios.
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]
t = ["_tarifa_teorica", ""]

# Directorios.
path_cfe = path + "CFE/"
dirs = [RCP[1]]
csv = ".csv"
path_data = [path + "data/" + x + "/data." for x in dirs]
f_path = []
for p in path_data:
    f_path = ( f_path + [p + str(x) for x in fut]
        + [p + str(x) + "_tarifa_teorica" for x in fut]
        + [p + str(x) + "_pob_pib" for x in fut] )
f_path = [ path + "data/Actual/data" + x for x in t ] + f_path

f_tarifa = "Tarifa_1.csv"
f_dac = "DAC.csv"
f_regiones = "Regiones.csv"

tarifa = pd.read_csv(path_cfe + f_tarifa)
dac = pd.read_csv(path_cfe + f_dac)
regiones = pd.read_csv(path_cfe + f_regiones)

# Tipos de escalones para las tarifas.
escalon_0 = tarifa["Escalon"].unique()

# Se establece la tarifa DAC para cada estado.
regiones[ ["DAC Propuesto", "Equilibrio"] ] = np.nan
for i in regiones.itertuples():
    regiones.loc[ i[0], ["DAC 04-2013", "Equilibrio"] ] = ( 
        dac.drop(8).loc[ dac["Region"] == i.Region,
        ["DAC 04-2013", "Equilibrio"] ].values[0] )

# Impuestos y cargos fijos.    
IVA = 1.16
#FIJO_DAC = 121.75
FIJO_DAC = 78.85
MULT = 0.86

In [ ]:
# Pago actual

cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
    "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

# Calculamos el pago para la tarifa
# teórica y para la tarifa real.
for k in [0, 1]:
    # Información mensual.
    df_0 = pd.read_csv(f_path[k] + csv)
    # Información anual.
    df_y = pd.read_csv(f_path[k] + "_yearly" + csv)

    # Promediamos el consumo y usuarios anuales.
    df = df_y.groupby("CVE_INEGI").mean()
    df["Tarifa"] = ""

    # Recuperamos la información de las tarifas.
    df[ [ "Tarifa", "NOM_ENT", "NOM_MUN" ] ] = (
        df_y[df_y["Año"] == 2010]
        .set_index("CVE_INEGI")[ 
        [ "Tarifa", "NOM_ENT", "NOM_MUN" ] ] )
    col = [x for x in df_y.columns]
    col.remove("CVE_INEGI")
    df = df[ col ]
    # Permite una tarifa diferente para cada año.
    #for i in df.itertuples():
    #    df.loc[i[0], "Tarifa"] = df_y.query(
    #        "CVE_INEGI==" + str(i[0]) +
    #        " & Año==2010" )["Tarifa"].values[0]

    # Redondeamos la cantidad de usuarios.
    df[["Usuarios_1*", "Usuarios_DAC"]] = df[
        ["Usuarios_1*", "Usuarios_DAC"]].round()

    # Retiramos la columna de año, que ya no tiene sentido.
    df.drop(columns = "Año", inplace = True)

    # Sumamos el consumo correspondiente al verano.
    df_2 = df_0.where(df_0["Verano"] == True)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum() / 7
    df[cons[0:2]] = df_2[["Consumo_1*", "Consumo_DAC"]]

    # Sumamos el consumo correspondiente al invierno.
    df_2 = df_0.where(df_0["Verano"] == False)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum() / 7
    df[cons[2:4]] = df_2[["Consumo_1*", "Consumo_DAC"]]

    # Retiramos las columnas de consumo total anual.
    df.drop(columns = [
    #"Consumo_1*",
    "Consumo_DAC"
    ], inplace = True)

    # Unimos las tarifas 1 y DAC.
    df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
    df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)
    df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
    df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)
    df["Usuarios_1*"] += df["Usuarios_DAC"]
    df.drop("Usuarios_DAC", axis = 1, inplace = True)

    df.reset_index(inplace = True)

    # Se crean columnas para los
    # escalones de verano e invierno.
    verano_escalon = []
    invierno_escalon = []
    for e in escalon_0:
        verano_escalon.append("Verano_" + e)
        invierno_escalon.append("Invierno_" + e)
    escalon = [verano_escalon,
        invierno_escalon]
    df[verano_escalon] = np.nan
    df[invierno_escalon] = np.nan

    # Se establece el valor de consumo límite para cada escalon.
    for i in df.itertuples():
        df.loc[i[0], invierno_escalon + verano_escalon] = tarifa.loc[
            tarifa["Tarifa"] == i.Tarifa, "Límite_p"].array * 6

    # Columnas para consumo en verano e invierno.
    estacion = ["Verano_", "Invierno_"]
    verano_escalon_cons = [ "Cons_" +
        x for x in verano_escalon ]
    invierno_escalon_cons = [ "Cons_" +
        x for x in invierno_escalon ]
    escalon_cons = [ verano_escalon_cons,
        invierno_escalon_cons ]

    # Columnas para verano.
    df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[0]] = df[escalon[0]].copy()
    df[estacion[0] + "Pago"] = 0

    # Columnas para invierno.
    df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[1]] = df[escalon[1]].copy()
    df[estacion[1] + "Pago"] = 0

    df["DAC_Pago"] = 0

    # Iteramos para verano e invierno.
    for e in [0, 1]:

        # Offset de columnas de verano e invierno.
        if e == 0: s = 0
        else: s = 6

        col = np.concatenate( [
            [estacion[e] + "Consumo_Usuario"],
            escalon_cons[e]
            ] )

        # Calculamos los escalones de consumo
        # de acuerdo con cada límite.
        for i in range(len(col[:-1])):
            df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

        # Eliminamos valores negativos.
        df[col[1:]] = df[col[1:]].clip( lower = 0 )

        col = np.concatenate( [ escalon_cons[e] ] )
        col = np.flip(col)

        # Recorremos los escalones a su posición adecuada.
        for i in range(len(col[:-1])):
            df[col[i]] = df[col[i + 1]]

        for i in df.itertuples():
            
            # No se está en tarifa DAC.
            if ( ( i.Verano_Consumo_Usuario +
                i.Invierno_Consumo_Usuario )
                <= ( i.Verano_Excedente * 2 ) ):
                # Si el consumo es menor al límite básico, el
                # escalon básico es igual al consumo total.
                if i[40 + s] == 0:
                    df.loc[i[0], escalon_cons[e][0]] = i[39 + s]
                # Si el consumo excede un escalón, 
                # el consumo de un escalón es su límite.
                for j in range(3):
                    if i[40 + s + j + 1] > 0:
                        df.loc[i[0], escalon_cons[e][j]] = i[
                            31 + s * 4 // 6 + j]
            
                df.loc[i[0], escalon_cons[e]] *= tarifa.query(
                  "Tarifa=='" + i.Tarifa + "' & Temporada=='"
                  + estacion[e][:-1] + "'")["Precio Propuesto"].array
        
            # Sí está en tarifa DAC.
            else:
                # El resto de pagos es igual a cero.
                df.loc[i[0], col] = 0

                # Se calcula el pago con todo el consumo general.
                df.loc[i[0], "DAC_Pago"] = ( (
                    i.Verano_Consumo_Usuario +
                    i.Invierno_Consumo_Usuario )
                    * regiones.loc[
                    regiones["CVE_ENT"] == i.CVE_ENT,
                    "DAC 04-2013"].values[0] + FIJO_DAC )
            
                #df.loc[i[0], "Tarifa"] = "DAC"

        # Se calcula el pago por temporada.
        df[estacion[e] + "Pago"] = ( df[
            np.concatenate( [ escalon_cons[e]
            ] ) ].sum(axis = 1) * IVA )

    # Se eliminan las columnas de apoyo.
    df.drop( columns = np.concatenate( [
        escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
        ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
        ] ), inplace = True )

    # Se calcula el pago total.
    df["Pago_real"] = ( df[[estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

    # Se eliminan los pagos parciales.
    df.drop( columns = [ estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

    # Unimos el consumo de verano e invierno.
    df["Consumo_1*"] = ( df["Verano_Consumo_1*"]
        + df["Invierno_Consumo_1*"] )
    df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
    df["Pago_sin_subsidio"] = np.nan

    # Calculamos el precio sin subsidio, es decir,
    # con un precio plano igual al precio marginal.
    for i in df.itertuples():
        df.loc[i[0], "Pago_sin_subsidio"] = ( 
            i.Consumo_Usuario * regiones.loc[
            regiones["CVE_ENT"] == i.CVE_ENT,
            "Equilibrio" ].values[0] * IVA * MULT )

    # Se calcula el subsidio.
    df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"] -
        df["Pago_real"] )
    df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

    df.to_csv(f_path[k] + "_Pago_Propuesto" + csv, index = False)

    print(f_path[k] + "_Pago_Propuesto" + csv)
    print(f"Pago    : {df['Pago_real'].sum():e}")
    print(f"Subsidio: {df['Subsidio'].sum():e}")
    print(f"Usuarios: {df['Usuarios_1*'].sum():e}")
    print(f"Consumo : {df['Consumo_1*'].sum():e}\n")  

df

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/Actual/data_tarifa_teorica_Pago_Propuesto.csv
Pago    : 3.738629e+06
Subsidio: 4.457490e+10
Usuarios: 3.322479e+07
Consumo : 5.323831e+10

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/Actual/data_Pago_Propuesto.csv
Pago    : 3.622745e+06
Subsidio: 4.825781e+10
Usuarios: 3.322479e+07
Consumo : 5.323831e+10



,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Tarifa,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Verano_Consumo_1*,Invierno_Consumo_1*,Pago_real,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio
0,1001,1.0,1.0,Aguascalientes,Aguascalientes,3.094919e+08,254974.0,1,-102.295872,21.811400,27.202617,13.115533,20.159075,43.460037,-49.675820,52.510950,-2.800402,0.998593,54.364928,845118.000000,1.374286e+11,23.691382,4.142857,7.245778,162160.723559,NaN,11.211926,116635.924759,1.605032e+08,1.489886e+08,1974.104948,1213.817321,3215.622943,1241.517994,3.165548e+08
1,1002,1.0,2.0,Aguascalientes,Asientos,1.243081e+07,11557.0,1,-102.045590,22.126479,26.473564,12.318128,19.395846,35.417511,-57.201100,55.798971,-4.047290,0.537158,41.414469,46075.142857,3.491166e+09,23.034981,4.142857,0.847945,75705.155679,NaN,11.211926,54337.426603,6.447211e+06,5.983595e+06,1491.598809,1075.608401,2849.482364,1357.883554,1.569306e+07
2,1003,1.0,3.0,Aguascalientes,Calvillo,1.809181e+07,16736.0,1,-102.704911,21.900640,27.019086,12.528155,19.773620,51.828022,-43.856953,46.549306,-1.860677,0.583761,61.132498,55283.142857,4.358511e+09,23.082358,4.142857,0.598719,78733.145869,NaN,11.211926,92335.783229,9.382205e+06,8.709606e+06,1510.462412,1081.011686,2863.796649,1353.334237,2.264940e+07
3,1004,1.0,4.0,Aguascalientes,CosÃ­o,4.038387e+06,3788.0,1,-102.297038,22.360619,26.583149,12.474749,19.528949,37.045483,-62.543280,58.554362,-6.007387,0.466796,47.664781,15363.000000,1.449504e+09,23.041425,4.142857,1.196649,94221.940180,NaN,11.211926,12838.354454,2.094395e+06,1.943991e+06,1458.403479,1066.099940,2824.292721,1365.889242,5.173988e+06
4,1005,1.0,5.0,Aguascalientes,JesÃºs MarÃ­a,3.476588e+07,30597.0,1,-102.445700,21.932112,26.906111,12.873946,19.890028,44.557915,-48.198683,48.564001,-2.633082,0.881738,53.936870,112079.000000,1.318374e+10,23.204577,4.142857,2.242050,117305.832917,NaN,11.211926,49989.509158,1.802838e+07,1.673750e+07,1703.310780,1136.251117,3010.135954,1306.825174,3.998493e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,Zacatecas,Villa Hidalgo,3.691307e+06,3923.0,1,-101.714324,22.407345,26.172767,12.650410,19.411589,29.787912,-58.152936,59.087596,-6.652236,1.244183,33.186718,18889.000000,1.104084e+09,23.055471,4.000000,0.508554,58432.484498,NaN,11.268627,37142.598662,2.025577e+06,1.665730e+06,1085.317892,940.939806,2451.615816,1366.297924,5.359987e+06
2465,32055,32.0,55.0,Zacatecas,Villanueva,8.254025e+06,8847.0,1,-102.851571,22.298130,26.540332,12.517529,19.528930,44.616968,-47.320534,49.280637,-3.686745,0.483920,50.683492,29902.000000,2.396876e+09,22.681947,4.142857,0.138660,80134.667679,NaN,11.268627,215650.560221,4.530459e+06,3.723565e+06,1066.869557,932.974438,2430.862076,1363.992519,1.206724e+07
2466,32056,32.0,56.0,Zacatecas,Zacatecas,4.321439e+07,37702.0,1,-102.678055,22.735730,25.742280,11.905951,18.824115,36.129680,-57.903010,55.026615,-6.280378,0.676493,44.517250,142958.571429,2.484715e+10,22.429208,4.142857,3.272704,173750.774044,NaN,11.268627,43682.101050,2.368850e+07,1.952589e+07,1738.076836,1146.209496,2986.445374,1248.368538,4.706599e+07
2467,32057,32.0,57.0,Zacatecas,Trancoso,2.057456e+07,15674.0,1,-102.312009,22.755625,26.267877,12.035475,19.151676,33.912960,-62.858518,61.301347,-6.292955,0.509237,44.236798,18421.428571,1.414295e+09,22.864307,4.000000,0.843291,76845.878314,NaN,11.268627,21844.700672,1.134779e+07,9.226769e+06,2319.161735,1312.655289,3420.119383,1100.957648,1.725641e+07


In [ ]:
# Pago futuro

names = ["", "_tarifa_teorica", "pob_pib"]

#for k in range(1, len(f_path)):
for k in range(2, len(f_path)):

    cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
        "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

    # Información mensual.
    df_0 = pd.read_csv(f_path[k] + csv)
    # Información anual.
    df = pd.read_csv(f_path[k] + "_yearly" + csv)
    df.set_index("CVE_INEGI", inplace = True)
    df["Usuarios_1*"] = df_0.groupby(
        "CVE_INEGI").mean()["Usuarios_1*"]
    #df.drop(columns = ["NOM_ENT", "NOM_MUN"], inplace = True)

    # Redondeamos la cantidad de usuarios.
    #df[["Usuarios_1*", "Usuarios_DAC"]] = df[
    #    ["Usuarios_1*", "Usuarios_DAC"]].round()
    df["Usuarios_1*"] = df["Usuarios_1*"].round()

    # Retiramos la columna de año, que ya no tiene sentido.
    df.drop(columns = "Año", inplace = True)

    # Sumamos el consumo correspondiente al verano.
    df_2 = df_0.where(df_0["Verano"] == True)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum()
    #df[cons[0:2]] = df_2[["Consumo_1*", "Consumo_DAC"]]
    df[cons[0]] = df_2["Consumo_1*"]

    # Sumamos el consumo correspondiente al invierno.
    df_2 = df_0.where(df_0["Verano"] == False)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum()
    #df[cons[2:4]] = df_2[["Consumo_1*", "Consumo_DAC"]]
    df[cons[2]] = df_2["Consumo_1*"]

    # Retiramos las columnas de consumo total anual.
    df.drop(columns = [
    #    "Consumo_1*",
        "Consumo_DAC"
        ], inplace = True)

    # Unimos las tarifas 1 y DAC.
    #df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
    #df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)
    #df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
    #df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)
    #df["Usuarios_1*"] += df["Usuarios_DAC"]
    df.drop("Usuarios_DAC", axis = 1, inplace = True)

    df.reset_index(inplace = True)
        
    # Se crean columnas para los
    # escalones de verano e invierno.
    verano_escalon = []
    invierno_escalon = []
    for e in escalon_0:
        verano_escalon.append("Verano_" + e)
        invierno_escalon.append("Invierno_" + e)
    escalon = [verano_escalon,
        invierno_escalon]
    df[verano_escalon] = np.nan
    df[invierno_escalon] = np.nan
        
    # Se establece el valor de consumo límite para cada escalon.
    for i in df.itertuples():
        df.loc[i[0], invierno_escalon + verano_escalon] = tarifa.loc[
            tarifa["Tarifa"] == i.Tarifa, "Límite_p"].array * 6

    # Columnas para consumo en verano e invierno.
    estacion = ["Verano_", "Invierno_"]
    verano_escalon_cons = [ "Cons_" +
        x for x in verano_escalon ]
    invierno_escalon_cons = [ "Cons_" +
        x for x in invierno_escalon ]
    escalon_cons = [ verano_escalon_cons,
        invierno_escalon_cons ]

    # Columnas para verano.
    df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[0]] = df[escalon[0]].copy()
    df[estacion[0] + "Pago"] = 0

    # Columnas para invierno.
    df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[1]] = df[escalon[1]].copy()
    df[estacion[1] + "Pago"] = 0

    df["DAC_Pago"] = 0

    # Iteramos para verano e invierno.
    for e in [0, 1]:

        # Offset de columnas de verano e invierno.
        if e == 0: s = 0
        else: s = 6

        col = np.concatenate( [
            [estacion[e] + "Consumo_Usuario"],
            escalon_cons[e]
            ] )

        # Calculamos los escalones de consumo
        # de acuerdo con cada límite.
        for i in range(len(col[:-1])):
            df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

        # Eliminamos valores negativos.
        df[col[1:]] = df[col[1:]].clip( lower = 0 )

        col = np.concatenate( [ escalon_cons[e] ] )
        col = np.flip(col)

        # Recorremos los escalones a su posición adecuada.
        for i in range(len(col[:-1])):
            df[col[i]] = df[col[i + 1]]

        for i in df.itertuples():
        
            # No se está en tarifa DAC.
            if ( ( i.Verano_Consumo_Usuario +
                i.Invierno_Consumo_Usuario )
                <= ( i.Verano_Excedente * 2 ) ):
                # Si el consumo es menor al límite básico, el
                # escalon básico es igual al consumo total.
                if i[40 + s] == 0:
                    df.loc[i[0], escalon_cons[e][0]] = i[39 + s]
                # Si el consumo excede un escalón, 
                # el consumo de un escalón es su límite.
                for j in range(3):
                    if i[40 + s + j + 1] > 0:
                        df.loc[i[0], escalon_cons[e][j]] = i[
                            31 + s * 4 // 6 + j]
                    
                df.loc[i[0], escalon_cons[e]] *= tarifa.query(
                    "Tarifa=='" + i.Tarifa + "' & Temporada=='"
                    + estacion[e][:-1] + "'")["Precio Propuesto"].array
    
            # Sí está en tarifa DAC.
            else:
                # El resto de pagos es igual a cero.
                df.loc[i[0], col] = 0

                # Se calcula el pago con todo el consumo general.
                df.loc[i[0], "DAC_Pago"] = ( (
                    i.Verano_Consumo_Usuario +
                    i.Invierno_Consumo_Usuario )
                    * regiones.loc[
                    regiones["CVE_ENT"] == i.CVE_ENT,
                        "DAC 04-2013"].values[0] + FIJO_DAC )
                    
                #df.loc[i[0], "Tarifa"] = "DAC"

        # Se calcula el pago por temporada.
        df[estacion[e] + "Pago"] = ( df[
            np.concatenate( [ escalon_cons[e]
            ] ) ].sum(axis = 1) * IVA )

    # Se eliminan las columnas de apoyo.
    df.drop( columns = np.concatenate( [
        escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
        ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
        ] ), inplace = True )

    # Se calcula el pago total.
    df["Pago_real"] = ( df[[estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

    # Se eliminan los pagos parciales.
    df.drop( columns = [ estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

    # Unimos el consumo de verano e invierno.
    df["Consumo_1*"] = ( df["Verano_Consumo_1*"]
        + df["Invierno_Consumo_1*"] )
    df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
    df["Pago_sin_subsidio"] = np.nan

    # Calculamos el precio sin subsidio, es decir,
    # con un precio plano igual al precio marginal.
    for i in df.itertuples():
        df.loc[i[0], "Pago_sin_subsidio"] = ( 
            i.Consumo_Usuario * regiones.loc[
            regiones["CVE_ENT"] == i.CVE_ENT,
            "Equilibrio" ].values[0] * IVA * MULT )

    # Se calcula el subsidio.
    df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"]
        - df["Pago_real"] )
    df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

    df.to_csv( f_path[k] + "_Pago_Propuesto" 
        + csv, index = False )

    print(f_path[k] + "_Pago_Propuesto" + csv)     
    print(f"Subsidio: {df.Subsidio.sum():e}")
    print(f"Usuarios: {df['Usuarios_1*'].sum():e}")
    print(f"Consumo : {df['Consumo_1*'].sum():e}\n")        

df

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2030_Pago_Propuesto.csv
Subsidio: 6.903200e+10
Usuarios: 4.624972e+07
Consumo : 7.577375e+10

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2050_Pago_Propuesto.csv
Subsidio: 8.398923e+10
Usuarios: 5.649878e+07
Consumo : 1.008955e+11

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2070_Pago_Propuesto.csv
Subsidio: 9.452654e+10
Usuarios: 6.185084e+07
Consumo : 1.203056e+11

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2030_tarifa_teorica_Pago_Propuesto.csv
Subsidio: 6.644516e+10
Usuarios: 4.624972e+07
Consumo : 7.577375e+10

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2050_tarifa_teorica_Pago_Propuesto.csv
Subsidio: 8.249676e+10
Usuarios: 5.649878e+07
Consumo : 1.008955e+11

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2070_tarifa_teorica_Pago_Propuesto.csv
Subsidio: 9.402075e+10
Usuarios: 6.185084e+07
Consumo

,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Tarifa,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Verano_Consumo_1*,Invierno_Consumo_1*,Pago_real,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio
0,1001,1,1,Aguascalientes,Aguascalientes,4.574505e+08,280148.0,1A,-102.295872,21.811400,26.831736,12.044003,19.437869,37.579649,-58.695830,39.397687,-1.554892,0.000000,33.053389,770385.185284,7.499149e+11,24.024788,5.0,6.605042,9.734286e+05,NaN,10.151,116635.924759,2.457641e+08,2.116865e+08,3059.890855,1632.888737,4325.819375,1265.928520,3.546473e+08
1,1002,1,2,Aguascalientes,Asientos,4.294597e+07,26289.0,1A,-102.045590,22.126479,26.175149,11.407515,18.791332,29.131229,-57.822357,45.202569,-1.999567,0.000000,24.865178,48773.242793,2.303653e+10,23.582296,5.0,0.897599,4.723190e+05,NaN,10.151,54337.426603,2.326719e+07,1.967878e+07,3062.408935,1633.610016,4327.730172,1265.321237,3.326403e+07
2,1003,1,3,Aguascalientes,Calvillo,3.993436e+07,31018.0,1A,-102.704911,21.900640,26.624624,11.371351,18.997987,42.775255,-56.057541,34.257481,-1.037798,0.000000,36.123884,62450.058477,3.000173e+10,23.260357,5.0,0.676336,4.804116e+05,NaN,10.151,92335.783229,2.132426e+07,1.861010e+07,1853.943457,1287.457509,3410.709197,1556.765741,4.828776e+07
3,1004,1,4,Aguascalientes,CosÃ­o,2.054145e+07,14556.0,1A,-102.297038,22.360619,26.324632,11.498821,18.911726,30.913884,-59.865402,45.637099,-2.373177,0.000000,26.391269,13824.881071,8.816139e+09,23.338507,5.0,1.076842,6.377009e+05,NaN,10.151,12838.354454,1.102188e+07,9.519564e+06,2285.950504,1411.201484,3738.529504,1452.579000,2.114374e+07
4,1005,1,5,Aguascalientes,JesÃºs MarÃ­a,9.520786e+07,57680.0,1A,-102.445700,21.932112,26.540400,11.733234,19.136817,36.261781,-57.062529,36.280679,-1.403318,0.001882,31.005882,100149.054521,7.796109e+10,23.525841,5.0,2.003401,7.784506e+05,NaN,10.151,49989.509158,5.084713e+07,4.436074e+07,3121.799272,1650.621767,4372.797396,1250.998124,7.215757e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32,54,Zacatecas,Villa Hidalgo,1.829533e+07,14180.0,1A,-101.714324,22.407345,25.849209,11.773135,18.811171,22.630819,-57.608295,46.918773,-3.764118,0.018831,19.301478,23832.883798,9.219849e+09,23.658661,4.0,0.641659,3.868541e+05,NaN,10.129,37142.598662,9.944730e+06,8.350601e+06,1863.590570,1290.220825,3361.666455,1498.075884,2.124272e+07
2465,32055,32,55,Zacatecas,Villanueva,1.744380e+07,14156.0,1A,-102.851571,22.298130,26.318819,11.516973,18.917895,36.665521,-56.640069,40.152447,-1.744371,0.000000,31.834995,25969.222288,1.671753e+10,22.734486,5.0,0.120423,6.437439e+05,NaN,10.129,215650.560221,9.572116e+06,7.871684e+06,1661.223602,1232.254888,3210.636382,1549.412780,2.193349e+07
2466,32056,32,56,Zacatecas,Zacatecas,1.268063e+08,85761.0,1A,-102.678055,22.735730,25.528542,10.854307,18.191425,31.804822,-59.470544,41.302565,-3.159900,0.005365,28.040103,98739.748932,1.490808e+11,21.999798,5.0,2.260417,1.509836e+06,NaN,10.129,43682.101050,7.114126e+07,5.566503e+07,2521.250878,1478.600856,3852.490055,1331.239177,1.141684e+08
2467,32057,32,57,Zacatecas,Trancoso,3.359958e+07,12037.0,1A,-102.312009,22.755625,26.021268,10.954921,18.488094,29.438527,-61.438575,44.840508,-3.524211,0.015802,25.823558,15014.074278,9.909378e+09,22.855076,4.0,0.687310,6.600059e+05,NaN,10.129,21844.700672,1.751753e+07,1.608205e+07,7104.267084,2791.358711,7272.876673,168.609589,2.029554e+06
